In [ ]:
`julia scripts/slurm/runners/runMrBayes.jl \
    -i data/sims/MSAs/gtdb-f__Burkholderiaceae-t128-l50000-b20.phy \
    -o _research/runMrBayes/gtdb-f__Burkholderiaceae-t128-l50000-b20 \
    -m WAG`

In [ ]:
`julia scripts/slurm/runners/runRAxML.jl \
    -i data/sims/MSAs/gtdb-f__Burkholderiaceae-t128-l50000-b20.phy \
    -o _research/runRAxML/gtdb-f__Burkholderiaceae-t128-l50000-b20 \
    -m WAG`

In [ ]:
`julia --threads=5 scripts/slurm/runners/runSPI_test.jl \
    -i data/sims/MSAs/gtdb-f__Burkholderiaceae-t128-l50000-b20.phy \
    -o _research/runSPI/gtdb-f__Burkholderiaceae-t128-l50000-b20`

In [1]:
using DrWatson
@quickactivate "Doran_etal_2022"
using SPI, SparseArrays, StatsBase
include(joinpath(srcdir(), "parsephylip.jl"))

_stringcolumntocharmtx (generic function with 1 method)

In [2]:
m = randn(5, 10)
usv = svd(m);

In [3]:
methods(calc_spi_mtx)

# 5 methods for generic function "calc_spi_mtx":
[1] calc_spi_mtx(A::Matrix{<:Number}; alpha, q) in SPI at /Users/ben/projects/Doran_etal_2022/src/SPI.jl/src/core.jl:65
[2] calc_spi_mtx(usv::SVD; alpha, q) in SPI at /Users/ben/projects/Doran_etal_2022/src/SPI.jl/src/core.jl:71
[3] calc_spi_mtx(usv::SVD, ::SPI.Left; alpha, q) in SPI at /Users/ben/projects/Doran_etal_2022/src/SPI.jl/src/core.jl:72
[4] calc_spi_mtx(usv::SVD, ::SPI.Right; alpha, q) in SPI at /Users/ben/projects/Doran_etal_2022/src/SPI.jl/src/core.jl:73
[5] calc_spi_mtx(vecs::Matrix{<:T}, vals::Vector{<:T}; alpha, q) where T<:Number in SPI at /Users/ben/projects/Doran_etal_2022/src/SPI.jl/src/core.jl:74

In [7]:
Adjoint{Float64, Matrix{Float64}} <: AbstractArray

true

In [7]:
calc_spi_mtx(usv, SPI.Left())

5×5 Matrix{Float64}:
 0.0      5.17492  7.9161   5.82748  7.54329
 5.17492  0.0      4.79148  4.6695   4.54546
 7.9161   4.79148  0.0      7.52756  9.17252
 5.82748  4.6695   7.52756  0.0      5.90755
 7.54329  4.54546  9.17252  5.90755  0.0

In [2]:
inputfile = joinpath(datadir(), "sims", "MSAs", "gtdb-f__Burkholderiaceae-t128-l50000-b20.phy")

"/Users/ben/projects/Doran_etal_2022/data/sims/MSAs/gtdb-f__Burkholderiaceae-t128-l50000-b20.phy"

In [3]:
@time phydf = readphylip(inputfile);

  0.579412 seconds (1.15 M allocations: 82.132 MiB, 2.63% gc time, 90.32% compilation time)


In [4]:
@time M = onehotencode(phydf.seqs);

  1.892307 seconds (5.00 M allocations: 522.334 MiB, 11.07% gc time, 71.34% compilation time)


In [5]:
M

128×113791 SparseMatrixCSC{Bool, Int64} with 6400000 stored entries:
⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛

In [7]:
@time vals, vecs = eigen(Matrix(M*M'));

  2.010717 seconds (2.10 M allocations: 166.098 MiB, 6.15% gc time, 41.00% compilation time)


In [8]:
@time dij = calc_spi_mtx(vecs, sqrt.(vals))
@time hc = hclust(dij, linkage=:average, branchorder=:optimal)
@time spitree = nwstr(hc, phydf.ids; labelinternalnodes=false)

  1.607017 seconds (6.67 M allocations: 343.290 MiB, 5.77% gc time, 99.89% compilation time)
  0.747150 seconds (1.60 M allocations: 83.857 MiB, 5.71% gc time, 99.96% compilation time)
  0.555643 seconds (1.38 M allocations: 71.176 MiB, 3.59% gc time, 99.98% compilation time)


"(((((Tip072:1.034791e+02,Tip071:1.034791e+02):2.561012e+02,((Tip068:3.957625e+01,Tip069:3.957625e+01):1.325768e+02,Tip070:1.721530e+02):1.874272e+02):3.788046e+02,((((((Tip073:4.580912e+01,Tip074:4.580912e+01):4.325944e+01,((Tip075:5.043481e+01,Tip076:5.043481e+01):8.78" ⋯ 3925 bytes ⋯ "Tip125:2.501657e+02):1.578581e+02,Tip126:4.080238e+02):6.882825e+01,((Tip122:9.779931e+01,Tip121:9.779931e+01):1.094509e+02,((Tip118:5.807020e+01,Tip119:5.807020e+01):2.600321e+01,Tip120:8.407341e+01):1.231768e+02):2.696018e+02):2.267859e+02):1.122570e+02):0.000000e+00;"

In [9]:
a = rand(5,5)

5×5 Matrix{Float64}:
 0.703919   0.0082424  0.118133    0.00840764  0.197865
 0.169532   0.0294637  0.307487    0.549461    0.647698
 0.400662   0.0480815  0.418762    0.213605    0.162414
 0.0973322  0.488601   0.908361    0.100303    0.075693
 0.402079   0.189385   0.00966809  0.347315    0.294348

In [10]:
a[:, [1,2,2]]

5×3 Matrix{Float64}:
 0.703919   0.0082424  0.0082424
 0.169532   0.0294637  0.0294637
 0.400662   0.0480815  0.0480815
 0.0973322  0.488601   0.488601
 0.402079   0.189385   0.189385